<a href="https://colab.research.google.com/github/piaistes/FashionGAN-2020/blob/main/FashionGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **FASHION GAN 2020**



This script was part of a thesis by Pia Jabs at the Institute for Marketing and Customer Insight, University of Hamburg (12/2020).
It is based on StyleGAN2 and partially consists of code snippets published by other artists, which are declared as such in the following. 
To be able to run this code, you need to have your folder with 128x128 images stored on your Google-Drive. I also recommend to have at least 50GB of memory available on your Google-Drive. 

##Set-Up

Change tensorflow version to 1, as StyleGAN2 is configured for only this.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


Load package 'time' to keep track of processing times.

In [ ]:
import time

Connect Colab to your Google Drive to access more memory and to save files permanentely (files in Colab will be deleted once the session is interrupted/closed).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Clone NVIDIA StyleGAN2 Github Repository.

In [ ]:
!git clone https://github.com/NVlabs/stylegan2

Cloning into 'stylegan2'...
remote: Enumerating objects: 93, done.
remote: Total 93 (delta 0), reused 0 (delta 0), pack-reused 93
Unpacking objects: 100% (93/93), done.


Check if repository was cloned successfully.

In [ ]:
!ls /content/drive/My\ Drive/stylegan2

datasets	 LICENSE.txt		 __pycache__	   run_training.py
dataset_tool.py  metrics		 README.md	   test_nvcc.cu
dnnlib		 preprocessedImages128	 run_generator.py  training
Dockerfile	 pretrained_networks.py  run_metrics.py
docs		 projector.py		 run_projector.py




---



##Training

Update path to directory of latest .pkl file to ensure transfer learning after training was interrupted. 

In [ ]:
%cd /content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f

/content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f


The following code is based on https://github.com/NVlabs/stylegan2

Load 128x128 Images from your Google-Drive to change format to tf.records (insert path where to store tfrecords and path to original image folder).

In [ ]:
start = time.perf_counter()
!python /content/drive/My\ Drive/stylegan2/dataset_tool.py create_from_images /content/drive/My\ Drive/stylegan2/datasets/V1TF /content/drive/My\ Drive/Data\ Masterarbeit/preprocessedV1
end = time.perf_counter()
print(f"Transformation of images into tf.records in {end - start:0.4f} seconds.")

Loading images from "/content/drive/My Drive/Data Masterarbeit/preprocessedV1"
Creating dataset "/content/drive/My Drive/stylegan2/datasets/V1TF"
Added 14434 images.


Training (insert data-directory of tfrecords, dataset = name of tfrecords dataset, path to result-directory).

In [ ]:
!python /content/drive/My\ Drive/stylegan2/run_training.py --num-gpus=1 --data-dir=/content/drive/My\ Drive/stylegan2/datasets --config=config-f \
  --dataset=V1TF --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1 --mirror-augment=true --metrics=none

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f
dnnlib: Running training.training_loop.training_loop() on localhost...
Streaming data using training.dataset.TFRecordDataset...
tcmalloc: large alloc 4294967296 bytes == 0x71f8000 @  0x7f97bed83001 0x7f97bb80f765 0x7f97bb873bb0 0x7f97bb875a4f 0x7f97bb90c048 0x50a7f5 0x50cfd6 0x507f24 0x509202 0x594b01 0x54a17f 0x5517c1 0x59fe1e 0x50d596 0x507f24 0x588fac 0x59fe1e 0x50d596 0x507f24 0x588fac 0x59fe1e 0x50d596 0x509918 0x50a64d 0x50c1f4 0x509918 0x50a64d 0x50c1f4 0x507f24 0x588fac 0x59fe1e
tcmalloc: large alloc 4294967296 bytes == 0x7f960e41e000 @  0x7f97bed811e7 0x7f97bb80f5e1 0x7f97bb873c78 0x7f97bb873f37 0x7f97bb90bf28 0x50a7f5 0x50cfd6 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24
tcmalloc: large alloc 429496

Calculate Metrics (insert path for data-directory for tfrecords, path to latest network pkl file, dataset = name of tfrecords dataset, path to result-directory).

In [ ]:
start = time.perf_counter()
!python /content/drive/My\ Drive/stylegan2/run_metrics.py --data-dir=/content/drive/My\ Drive/stylegan2/datasets --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00008-stylegan2-V1TF-1gpu-config-f/network-snapshot-000000.pkl --metrics=fid50k --dataset=V1TF --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1
end = time.perf_counter()
print(f"Calculating Metrics in {end - start:0.4f} seconds.")

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/00050-run-metrics
dnnlib: Running run_metrics.run() on localhost...
Evaluating metrics "fid50k" for "/content/drive/My Drive/Data Masterarbeit/resultsV1/00008-stylegan2-V1TF-1gpu-config-f/network-snapshot-000000.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
tcmalloc: large alloc 4294967296 bytes == 0x583ea000 @  0x7f207ba6c001 0x7f20784f8765 0x7f207855cbb0 0x7f207855ea4f 0x7f20785f5048 0x50a7f5 0x50cfd6 0x507f24 0x509202 0x594b01 0x54a17f 0x5517c1 0x59fe1e 0x50d596 0x507f24 0x588fac 0x59fe1e 0x50d596 0x509918 0x50a64d 0x50c1f4 0x58e809 0x4b5d7f 0x50c467 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x509202 0x594b01
tcmalloc: large alloc 4294967296 bytes == 0x7f1e66000000 @  0x7f207ba6a1e7 0x7f20784f85e1 0x7f207855cc78 0x7f207855cf37 0x7f20785f4f28 0x50a7f5 0x50cfd6 0x507f24 0x509c50 0x50

Generate 100 Images (insert insert path to network pkl file, path to result directory).

In [ ]:
!python /content/drive/My\ Drive/stylegan2/run_generator.py generate-images --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl --seeds=0-99 --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/GeneratedImages99

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/GeneratedImages99/00002-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 0 (0/100) ...
Generating image for seed 1 (1/100) ...
Generating image for seed 2 (2/100) ...
Generating image for seed 3 (3/100) ...
Generating image for seed 4 (4/100) ...
Generating image for seed 5 (5/100) ...
Generating image for seed 6 (6/100) ...
Generating image for seed 7 (7/100) ...
Generating image for seed 8 (8/100) ...
Generating image for seed 9 (9/100) ...
Generating image for seed 10 (10/100) ...
Generating image for seed 11 (11/100)

## `Manipulation`

Change pathes in the following accordingly to training.

Change Truncation-Psi for selected Images.

In [ ]:
!python /content/drive/My\ Drive/stylegan2/run_generator.py generate-images --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl --seeds=21 --truncation-psi=1 --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/GeneratedImages99

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/GeneratedImages99/00007-generate-images
dnnlib: Running run_generator.generate_images() on localhost...
Loading networks from "/content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating image for seed 21 (0/1) ...
dnnlib: Finished run_generator.generate_images() in 14s.


Transformation of real images into tf.records. 



In [ ]:
!python /content/drive/My\ Drive/stylegan2/dataset_tool.py create_from_images /content/drive/My\ Drive/stylegan2/datasets/V1ProjektionTF /content/drive/My\ Drive/Data\ Masterarbeit/V1Projektion

Loading images from "/content/drive/My Drive/Data Masterarbeit/V1Projektion"
Creating dataset "/content/drive/My Drive/stylegan2/datasets/V1ProjektionTF"
Added 3 images.


Projection of real images in W.

In [ ]:
!python /content/drive/My\ Drive/stylegan2/run_projector.py project-real-images --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl \
  --dataset=V1ProjektionTF --data-dir=/content/drive/My\ Drive/stylegan2/datasets --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/Projektion

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/Projektion/00002-project-real-images
dnnlib: Running run_projector.project_real_images() on localhost...
Loading networks from "/content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Loading images from "V1ProjektionTF"...
tcmalloc: large alloc 4294967296 bytes == 0x56812000 @  0x7f0f32d93001 0x7f0f308b7765 0x7f0f3091bbb0 0x7f0f3091da4f 0x7f0f309b4048 0x50a7f5 0x50cfd6 0x507f24 0x509202 0x594b01 0x54a17f 0x5517c1 0x59fe1e 0x50d596 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x507f24 0x588fac 0x59fe1e 0x50d596 0x509918 0x50a64d 0x50c1f4 0x509918 0x50a64d 0x50c1f4 0x507f24 0x588fac 0x59fe1e
tcmalloc: large alloc 4294967296 bytes == 0x7f0d05c40000 @  0x7f0f32d911e7 0x7f0f308b75e1 0x7f0f3091bc78 0x7

Interpolation  of generated Images.

In [ ]:
!python run_generator.py generate-latent-walk --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl --walk-type='line-w' --seeds=27,28 --frames 10 --truncation-psi=0.5

Style-Mixing for generated Images.

In [ ]:
!python /content/drive/My\ Drive/stylegan2/run_generator.py style-mixing-example --network=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl --row-seeds=27,9,69,71,81 --col-seeds=28,39,72,75,85 --col-styles=0,1,2,3,8,9,10,11 --result-dir=/content/drive/My\ Drive/Data\ Masterarbeit/resultsV1/Style 

Local submit - run_dir: /content/drive/My Drive/Data Masterarbeit/resultsV1/Style/00033-style-mixing-example
dnnlib: Running run_generator.style_mixing_example() on localhost...
Loading networks from "/content/drive/My Drive/Data Masterarbeit/resultsV1/00022-stylegan2-V1TF-1gpu-config-f/network-snapshot-000024.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...
dnnlib: Finished run_generator.style_mixing_example() in 13s.


The following code is based on https://colab.research.google.com/drive/1qGU6NCz-2SI1BSLkFTZq0gYN2DNnXIXm#scrollTo=9oR65UOnNfV- - to additionally allow for manipulation of projected (real) images.

In [ ]:
%cd /content/drive/My\ Drive/stylegan2

/content/drive/My Drive/stylegan2


In [ ]:
!pip install opensimplex

Insert id to named pkl file (retrieve from Google-Drive Link of .pkl File)

In [ ]:
!mkdir pkl
!gdown --id 1gbxwfHNOaGjGsLNTmmSrNA85X2VWHHOq -O /content/drive/My\ Drive/stylegan2/pkl/vgg16_zhang_perceptual.pkl

mkdir: cannot create directory ‘pkl’: File exists
Downloading...
From: https://drive.google.com/uc?id=1gbxwfHNOaGjGsLNTmmSrNA85X2VWHHOq
To: /content/drive/My Drive/stylegan2/pkl/vgg16_zhang_perceptual.pkl
58.9MB [00:00, 95.8MB/s]


Insert path to pkl file.

In [ ]:
network_pkl = "/content/drive/My Drive/Data Masterarbeit/resultsV3/00048-stylegan2-V3TF-1gpu-config-f/network-snapshot-000008.pkl"

import argparse
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import numpy as np
from math import ceil
from PIL import Image, ImageDraw
import imageio

import pretrained_networks

# Convert uploaded images to TFRecords
import dataset_tool

# Run the projector
import training.dataset
import training.misc
import projector
import os 

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def interpolate(zs, steps,type='linear'):
  out = []
  for i in range(len(zs)-1):
    c = zs[i+1]-zs[i]

    for index in range(steps):
      fraction = index/float(steps) # t/d
      
      # translated from: https://github.com/danro/jquery-easing/blob/master/jquery.easing.js
      # see https://easings.net/ for examples
      if type == 'linear':
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
      elif type == 'easeInSine':
        out.append( -c * np.cos(fraction * (np.pi/2)) + c + zs[i] ) # -c * Math.cos(t/d * (Math.PI/2)) + c + b
      elif type == 'easeOutSine':
        out.append( c * np.sin(fraction * (np.pi/2)) + zs[i]) # c * Math.sin(t/d * (Math.PI/2)) + b
      elif type == 'easeInOutSine':
        out.append(-c/2 * (np.cos(np.pi*fraction) - 1.0) + zs[i]) # -c/2 * (Math.cos(Math.PI*t/d) - 1) + b;
      elif type == 'easeInQuad':
        out.append(c * fraction * fraction + zs[i]) # c*(t/=d)*t + b;
      elif type == 'easeOutQuad':
       out.append(-c * fraction * (fraction-2) + zs[i]) # -c *(t/=d)*(t-2) + b;
      # elif type == 'easeInOutQuad':
      #   if(fraction/2 < 1):
      #     out.append( ((c/2)*fraction*fraction) + zs[i]) #if ((t/=d/2) < 1) return c/2*t*t + b;
      #   else:
		  #     out.append( (-c/2) * ((index-=1)*(index-2) - 1) + zs[i]; #return -c/2 * ((--t)*(t-2) - 1) + b;
      else: 
        out.append( c * fraction + zs[i] ) # c*(t/d)+b
  return out

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location + ('%05d.png' % (idx))
    img.save(file)

def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in enumerate(dlatents):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs  

print('Loading networks from "%s"...' % network_pkl)
_G, _D, Gs = pretrained_networks.load_networks(network_pkl)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

Loading networks from "/content/drive/My Drive/Data Masterarbeit/resultsV3/00048-stylegan2-V3TF-1gpu-config-f/network-snapshot-000008.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Loading... Done.


In [ ]:
!mkdir projection
!mkdir projection/imgs
!mkdir projection/out

mkdir: cannot create directory ‘projection’: File exists
mkdir: cannot create directory ‘projection/imgs’: File exists
mkdir: cannot create directory ‘projection/out’: File exists


In [ ]:
# Convert uploaded images to TFRecords
dataset_tool.create_from_images("./projection/records/", "./projection/imgs/", True)

def project_image_run(proj, targets, png_prefix, num_snapshots):
    snapshot_steps = set(proj.num_steps - np.linspace(0, proj.num_steps, num_snapshots, endpoint=False, dtype=int))
    training.misc.save_image_grid(targets, png_prefix + 'target.png', drange=[-1,1])
    proj.start(targets)
    while proj.get_cur_step() < proj.num_steps:
        print('\r%d / %d ... ' % (proj.get_cur_step(), proj.num_steps), end='', flush=True)
        proj.step()
        if proj.get_cur_step() in snapshot_steps:
            training.misc.save_image_grid(proj.get_images(), png_prefix + 'step%04d.png' % proj.get_cur_step(), drange=[-1,1])
    print('\r%-30s\r' % '', end='', flush=True)
    type(proj.get_noises())
    #if you want to do interpolations, name the file below something memorable
    file_name = '3'
    np.save(('./projection/'+file_name+'.npy'), proj.get_dlatents())

def project_real_images(dataset_name, data_dir, num_images, num_snapshots):
    proj = projector.Projector()
    proj.set_network(Gs)
    #num_steps = how many iterations; larger = ~more accurate but longer run times 
    proj.num_steps=1500

    print('Loading images from "%s"...' % dataset_name)
    dataset_obj = training.dataset.load_dataset(data_dir=data_dir, tfrecord_dir=dataset_name, max_label_size=0, verbose=True, repeat=False, shuffle_mb=0)
    print(dataset_obj.shape, Gs.output_shape[1:])
    assert dataset_obj.shape == Gs.output_shape[1:]

    for image_idx in range(num_images):
        print('Projecting image %d/%d ...' % (image_idx, num_images))
        images, _labels = dataset_obj.get_minibatch_np(1)
        images = training.misc.adjust_dynamic_range(images, [0, 255], [-1, 1])
        project_image_run(proj, targets=images, png_prefix=dnnlib.make_run_dir_path('projection/out/image%04d-' % image_idx), num_snapshots=num_snapshots)

project_real_images("records","./projection",1,10)

Loading images from "./projection/imgs/"
Creating dataset "./projection/records/"
Added 1 images.
Loading images from "records"...
Streaming data using training.dataset.TFRecordDataset...
Dataset shape = [3, 256, 256]
Dynamic range = [0, 255]
Label size    = 0
[3, 256, 256] [3, 256, 256]
Projecting image 0/1 ...
0 / 1500 ... 

Interpolation of projected images.


In [ ]:
latent1 = np.load('/content/drive/My Drive/stylegan2/projection/3.npy')
latent2 = np.load('/content/drive/My Drive/stylegan2/projection/4.npy')

imgs = generate_images_in_w_space(interpolate([latent1,latent2],10,'linear'),0.5)

!rm -rf interpolations
%mkdir interpolations
saveImgs(imgs,'./interpolations/')